<a href="https://colab.research.google.com/github/Carlo-pien/Projet_ML_2_MA1/blob/master/Experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Experiments with unet model
## Initializations

In [ ]:
!pip install einops
!pip install pytorch-lightning
!pip install -q -U segmentation-models-pytorch albumentations > /dev/null

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

In [ ]:
%matplotlib inline
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os,sys
from PIL import Image
import numpy as np
import math
from sklearn.model_selection import KFold
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset
from torch.utils.data import TensorDataset, DataLoader
from torchvision.io import read_image
from torchvision import transforms as transforms
import albumentations.augmentations.transforms as tf
from torchvision.transforms import Compose
import albumentations as A
import torchvision.transforms.functional as TF
from einops import rearrange, reduce
import pytorch_lightning as pl
import cv2
import os, cv2
import numpy as np
import pandas as pd
import random, tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import warnings
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
import pytorch_lightning as pl
warnings.filterwarnings("ignore")

torch.manual_seed(1)

## System path definition

In [ ]:
import sys
sys.path.append('drive/MyDrive/Projet_ML/') 
#from Unet_model import Unet
from Unet_model_lightning import Lit_Net

## Helper functions

In [ ]:
# Helper functions

def load_image(infilename):
    data = mpimg.imread(infilename)
    return data

def img_float_to_uint8(img):
    rimg = img - np.min(img)
    rimg = (rimg / np.max(rimg) * 255).round().astype(np.uint8)
    return rimg

# Concatenate an image and its groundtruth
def concatenate_images(img, gt_img):
    nChannels = len(gt_img.shape)
    w = gt_img.shape[0]
    h = gt_img.shape[1]
    if nChannels == 3:
        cimg = np.concatenate((img, gt_img), axis=1)
    else:
        gt_img_3c = np.zeros((w, h, 3), dtype=np.uint8)
        gt_img8 = img_float_to_uint8(gt_img)          
        gt_img_3c[:,:,0] = gt_img8
        gt_img_3c[:,:,1] = gt_img8
        gt_img_3c[:,:,2] = gt_img8
        img8 = img_float_to_uint8(img)
        cimg = np.concatenate((img8, gt_img_3c), axis=1)
    return cimg

def to_RGB(mask):
  return np.repeat(np.round(mask)[:, :, np.newaxis], 3, axis=2)

def img_crop(im, w, h):
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    is_2d = len(im.shape) < 3
    for i in range(0,imgheight,h):
        for j in range(0,imgwidth,w):
            if is_2d:
                im_patch = im[j:j+w, i:i+h]
            else:
                im_patch = im[j:j+w, i:i+h, :]
            list_patches.append(im_patch)
    return list_patches

# Data loading and preparation

In [ ]:
root_dir = "/content/drive/MyDrive/Projet_ML/training/"
n = 100
image_dir = root_dir + "images/"
files = os.listdir(image_dir)
n = min(n, len(files))
print("Loading " + str(n) + " images")
imgs = [load_image(image_dir + files[i]) for i in range(n)]
print(files[0])

gt_dir = root_dir + "groundtruth/"
print("Loading " + str(n) + " images")
gt_imgs = [load_image(gt_dir + files[i]) for i in range(n)]
print(files[0])

# Defining dataset and dataloaders


In [ ]:
class CustomDataset(Dataset):
    def __init__(self,images_np =  imgs, masks_np = gt_imgs, transform=None):
        self.transform = transform
        self.num_samples = np.array(images_np).shape[0]
        self.images_np = images_np
        self.masks_np = masks_np
        
    def __getitem__(self, index):        
        if self.transform:
            
            img1 = np.array(self.images_np)[index,:,:,:]
            msk1 = np.array(self.masks_np)[index,:,:]

            #Add a dimension to masks and reshape it to have correct input for transformation
            mask1 = np.expand_dims(msk1, axis = 0)
            
            mask1 = (rearrange(mask1, 'c h w -> h w c'))

            #Applying the transformations
            transformed = self.transform(image=img1, mask=mask1)
            image11 = rearrange(transformed['image'], 'h w c -> c h w')
            mask11 = rearrange(transformed['mask'], 'h w c -> c h w')
            mask11 = np.squeeze(mask11, axis = 0)
          
        return image11, mask11
    
    def __len__(self):
        return self.num_samples

### Defining our transformations

Our dataset in a small dataset, hence it quickly became obivous that we had to resort to data augmentation techniques to create a more robust model. Initially, we tought about using a RandomSearch, GridSearch or even cross-validation to validate different transformations.

Unfortunately, all of these methods required to high computation power for our means. On top of that, we'd have to optimize the other parameters such as learning rate or u-net level at the same time. This prooved infeasible to do within the given time and computational power.

To test the transformations, we defined three groups:
1.  A baseline group, in which we only use horizontal and vertical flips. These transformations do not alter the road geometry and are as such usefull.
2.  A geometric transformations group, in which we applied multiple transformations that are of geomtric nature : rotations, rescaling, "Zooming" (cropping and resizing), shearing. 
3.  A non-geometric transformations group, in which we applied multiple transformations that are of non-geomtric nature : Noise (gaussian noise), blur and compression
3.  A mixed transformations group, in which we applied both geometric and non-geometric transformations. We however did not use shearing and "Zooming" transformations as these empirically lead to worse results. 


With more time, we would have liked to explore these transformations more in-depth with a greater variety of groups. Also, it would be interresting to tune the hyperparameters of these transformations : probability, cropping size, noise level, shearing dimenstions...

In [ ]:
#Definition of the different transformations

#Baseline
train_transform1 = [
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
  ]
my_transform1 = A.Compose(train_transform1)


#Geometric transformations
train_transform2 = [
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomSizedCrop(min_max_height=	[200,200], height = 400, width = 400, p = 0.5),
        A.ShiftScaleRotate(p=0.5),
        A.Affine(shear=[-10, 10], scale=1.1, fit_output=False, p=.1),
    ]
my_transform2 = A.Compose(train_transform2)


#Non-geometric transformations        
train_transform3 = [
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.GaussNoise(var_limit=(.001, .03), p=.5),
        A.Blur(blur_limit=5., p=.2),
        A.ImageCompression(quality_lower=50, p=.2),
    ]
my_transform3 = A.Compose(train_transform3)


#mixed transformations
train_transform4 = [
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.ShiftScaleRotate(p=0.5),
        A.GaussNoise(var_limit=(.001, .03), p=.5),
        A.Blur(blur_limit=5., p=.2),
        A.ImageCompression(quality_lower=50, p=.2),
    ]
my_transform4 = A.Compose(train_transform4)


### Creating the train and validation dataloaders

After research and testing on the subject, we arrived at the conclusion that implemeting a cross-validation to optimize our hyper parameters is not a viable option. Indeed, the requierd computation power is well above our possibilities.

As such, we decided an alternative approach : define a small and random validation set to tune the parameters to some extent. We shall be carefull as the variance of this validation may be high and we shall be vary as not to overfit the validation set. 

In [ ]:
batch_size = 4
validation_split = .15
shuffle_dataset = True
random_seed= 1

### Training the 4 models

Note : we used only 30 epochs for these models as we could not afford to wait 10h+ for all models to train with 150-200 epochs.

**These 4 cells below do not need to be run !** The models are saved in the drive and are reloaded in the cells further below

In [ ]:
#NO NEED TO RUN THE CELL, MODEL IS SAVED AND RELOADED LATER

#initializing the dataset and associated dataloaders for baseline transformations group
dataset = CustomDataset(transform=my_transform1)

# Creating data indices for training and validation splits:
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler)

#train the model for transforms1
l_unet = Lit_Net(in_chans=3, out_chans=1)
trainer = pl.Trainer(max_epochs = 30, progress_bar_refresh_rate=1, gpus=1, auto_scale_batch_size=True, accumulate_grad_batches=1)
trainer.fit(l_unet, train_dataloaders = train_loader)

#Saving the model
path = os.path.join('drive/MyDrive/Projet_ML/trained_models/unet_30epoch_transformations1.pth')
torch.save(l_unet, path)

In [ ]:
#NO NEED TO RUN THE CELL, MODEL IS SAVED AND RELOADED LATER

#initializing the dataset and associated dataloaders for geometric transformations group
dataset = CustomDataset(transform=my_transform2)

# Creating data indices for training and validation splits:
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler)

#train the model for transforms1
l_unet = Lit_Net(in_chans=3, out_chans=1)
trainer = pl.Trainer(max_epochs = 30, progress_bar_refresh_rate=1, gpus=1, auto_scale_batch_size=True, accumulate_grad_batches=1)
trainer.fit(l_unet, train_dataloaders = train_loader)

#Saving the model
path = os.path.join('drive/MyDrive/Projet_ML/trained_models/unet_30epoch_transformations2.pth')
torch.save(l_unet, path)

In [ ]:
#NO NEED TO RUN THE CELL, MODEL IS SAVED AND RELOADED LATER

#initializing the dataset and associated dataloaders for non-geometric transformations group
dataset = CustomDataset(transform=my_transform3)

# Creating data indices for training and validation splits:
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler)

#train the model for transforms1
l_unet = Lit_Net(in_chans=3, out_chans=1)
trainer = pl.Trainer(max_epochs = 30, progress_bar_refresh_rate=1, gpus=1, auto_scale_batch_size=True, accumulate_grad_batches=1)
trainer.fit(l_unet, train_dataloaders = train_loader)

#Saving the model
path = os.path.join('drive/MyDrive/Projet_ML/trained_models/unet_30epoch_transformations3.pth')
torch.save(l_unet, path)

In [ ]:
#NO NEED TO RUN THE CELL, MODEL IS SAVED AND RELOADED LATER

#initializing the dataset and associated dataloaders for mixed transformations group
dataset = CustomDataset(transform=my_transform4)

# Creating data indices for training and validation splits:
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler)

#train the model for transforms1
l_unet = Lit_Net(in_chans=3, out_chans=1)
trainer = pl.Trainer(max_epochs = 30, progress_bar_refresh_rate=1, gpus=1, auto_scale_batch_size=True, accumulate_grad_batches=1)
trainer.fit(l_unet, train_dataloaders = train_loader)


path = os.path.join('drive/MyDrive/Projet_ML/trained_models/unet_30epoch_transformations4.pth')
torch.save(l_unet, path)

# Evalutating the models

### helper functions:

In [ ]:
from sklearn.metrics import f1_score

#Threshold function, which always thresholds an image with the given threshold value
def create_threshold(threshold):
    return lambda pred : pred > threshold

#Applies a threshold function to the predictions, and computes the f1 score with respect to the target images
def compute_score_thresholded(threshold_func, preds, target):
    thresholded = np.stack([threshold_func(sigmoid_v(pred)) for pred in preds])
    return f1_score(np.ravel(target), np.ravel(thresholded.astype(int)))

#Selects the best threshold with respect to f1-score from the given predictions and target images
def select_best_threshold(threshold_funcs, preds, target):
    assert len(preds) == len(target)
    best_func = None
    best_thresh = 0
    best_score = 0
    for threshold, threshold_func in threshold_funcs:
        score = compute_score_thresholded(threshold_func, preds, target)
        if score > best_score:
            best_thresh = threshold
            best_score = score
            best_func = threshold_func
    return best_func, best_thresh, best_score

#used for iterating on dataloader
def cycle(iterable):
    while True:
        for x in iterable:
            yield x

# custom sigmoid function
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

# define vectorized sigmoid
sigmoid_v = np.vectorize(sigmoid)

#initialize the thresholds
THRESHOLDS = [(x,create_threshold(x)) for x in np.linspace(0.1,0.7,num=20)]

# Reloading datasets and dataloaders

To evaluate the model, we must reload all dataloaders and datasets. While this is not optimal in terms of coding as all these where initialized in earlier cells, we must proceed this way to save RAM memory and save GPU power.

Indeed, after training the models we saved them onto our drive and can now reload them. This way, this second part is effectively segmented from the training which enables us to run it separately.


In [ ]:
#Evalutating model with baseline transformations
dataset = CustomDataset(transform=my_transform1)

#Creating data indices for training and validation splits:
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler)


path = os.path.join('drive/MyDrive/Projet_ML/trained_models/unet_30epoch_transformations1.pth')
model = torch.load(path)
model.eval()
val_iter = iter(cycle(validation_loader))

preds = []
targets = []
for i in range(len(val_indices)):
  print("iter", i)
  data = next(val_iter)
  x,y = data
  preds.append(model(x)[0].detach().numpy())
  targets.append(y[0].detach().numpy())

best_func1, best_thresh1, best_score1 = select_best_threshold(THRESHOLDS, np.array(preds), np.round(np.array(targets)))
print("Best score for baseline transformations :",best_score1) 

In [ ]:
#Evalutating model with geometric transformations

dataset = CustomDataset(transform=my_transform2)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler)


path = os.path.join('drive/MyDrive/Projet_ML/trained_models/unet_30epoch_transformations2.pth')
model = torch.load(path)
model.eval()
val_iter = iter(cycle(validation_loader))

preds = []
targets = []
for i in range(len(val_indices)):
  print("iter", i)
  data = next(val_iter)
  x,y = data
  preds.append(model(x)[0].detach().numpy())
  targets.append(y[0].detach().numpy())

best_func2, best_thresh2, best_score2 = select_best_threshold(THRESHOLDS, np.array(preds), np.round(np.array(targets)))
print("Best score for geometric transformations :",best_score2) 

In [ ]:
#Evalutating model with geometric transformations

dataset = CustomDataset(transform=my_transform3)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler)


path = os.path.join('drive/MyDrive/Projet_ML/trained_models/unet_30epoch_transformations3.pth')
model = torch.load(path)
model.eval()
val_iter = iter(cycle(validation_loader))

preds = []
targets = []
for i in range(len(val_indices)):
  print("iter", i)
  data = next(val_iter)
  x,y = data
  preds.append(model(x)[0].detach().numpy())
  targets.append(y[0].detach().numpy())

best_func3, best_thresh3, best_score3 = select_best_threshold(THRESHOLDS, np.array(preds), np.round(np.array(targets)))
print("Best score for non-geometric transformations :",best_score3) 

In [ ]:
#Evalutating model with geometric transformations

dataset = CustomDataset(transform=my_transform4)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler)


path = os.path.join('drive/MyDrive/Projet_ML/trained_models/unet_30epoch_transformations4.pth')
model = torch.load(path)
model.eval()
val_iter = iter(cycle(validation_loader))

preds = []
targets = []
for i in range(len(val_indices)):
  print("iter", i)
  data = next(val_iter)
  x,y = data
  preds.append(model(x)[0].detach().numpy())
  targets.append(y[0].detach().numpy())

best_func4, best_thresh4, best_score4 = select_best_threshold(THRESHOLDS, np.array(preds), np.round(np.array(targets)))
print("Best score for mixed transformations :",best_score4) 

In [ ]:
print("Best score for baseline transformations :", best_score1,"\n", "Best score for geometric transformations :", best_score2, "\n", 
      "Best score for non-geometric transformations :", best_score3, "\n", "Best score for mixed transformations :", best_score4)

  As expected, geometric transformations yield the worse results, as these do not preserve the geometry of the roads. Cropping and shearing may results in a loss of information about the links between the pixels, which plays an important role in unets and in the task at hand.

  It may seem surprising that the best evaluation here is the baseline model with very few transformations. This is likely due to the fact that this model is "easier" to train as there are less different pictures. It therefore gives the best results when trained with only 30 epochs. As mentionned earlier, due to time limits and Google colab GPU usage limits we could not train these models with a higher number of epochs.

  Empirically, the models that worked best were mixed models, without cropping or shearing. This results can be observed here.